In [1]:
import numpy as np
import pandas as pd

In [2]:
import scipy.io

In [3]:
data = scipy.io.loadmat('/Users/Sophie/Library/CloudStorage/OneDrive-Personnel/IP/Data Stream Processing/PySAD_to_River/data/arrhythmia.mat') 

In [4]:
X = data['X']
y = data['y']

In [5]:
from anomaly.base import AnomalyDetector

## Import our model

In [6]:
from river import metrics

In [21]:
from models.xstream import xStream

In [15]:
pip install pysad

     |████████████████████████████████| 73 kB 4.2 MB/s eta 0:00:011
     |████████████████████████████████| 141 kB 22.6 MB/s eta 0:00:01
  Created wheel for pyod: filename=pyod-1.0.6-py3-none-any.whl size=175085 sha256=29b417165bce37ef2f26b5cc154e2e4633380243a64db998cb13f15226256794
  Stored in directory: /Users/Sophie/Library/Caches/pip/wheels/98/93/e6/6d40410d9635ecde42d06041a1ba7f2ee7396e036fcf702e73
Successfully built pyod
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install combo

  Created wheel for combo: filename=combo-0.1.3-py3-none-any.whl size=42871 sha256=149bdd0cd9c0f93168da1d2bab08d37a77a27d44c2e4aa3715b8a0f5485b0c55
  Stored in directory: /Users/Sophie/Library/Caches/pip/wheels/ec/44/39/0667fea44a2dfe692cc2a51f0f79ea49b9dee7def53594ef2e
Successfully built combo
Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install mmh3

Note: you may need to restart the kernel to use updated packages.


In [8]:
from pysad.transform.probability_calibration import ConformalProbabilityCalibrator

In [9]:
from river.stream import iter_array

In [10]:
np.random.seed(61)

In [11]:
model = xStream()
calibrator = ConformalProbabilityCalibrator(windowed=True, window_size=300)
i=1
for xi, yi in iter_array(X,y):
    i += 1
    x_array = np.array(list(xi.values()))
    y_array = np.array(list(yi.values()))
    anomaly_score = model.score_one(x_array)
    model = model.learn_one(x_array)
#    anomaly_score = model.learn_score_one(x_array)
    
    calibrated_score = calibrator.fit_transform(anomaly_score)  # Fit & calibrate score.

    # Output if the instance is anomalous.
    if calibrated_score > 0.95:  # If probability of being normal is less than 5%.
        print(f"Alert: {i} data point is anomalous.")
#        print(xi)

Alert: 26 data point is anomalous.
Alert: 51 data point is anomalous.
Alert: 52 data point is anomalous.
Alert: 53 data point is anomalous.
Alert: 56 data point is anomalous.
Alert: 59 data point is anomalous.
Alert: 63 data point is anomalous.
Alert: 64 data point is anomalous.
Alert: 67 data point is anomalous.
Alert: 74 data point is anomalous.
Alert: 83 data point is anomalous.
Alert: 136 data point is anomalous.
Alert: 150 data point is anomalous.
Alert: 202 data point is anomalous.
Alert: 226 data point is anomalous.
Alert: 229 data point is anomalous.
Alert: 230 data point is anomalous.
Alert: 232 data point is anomalous.
Alert: 234 data point is anomalous.
Alert: 235 data point is anomalous.
Alert: 238 data point is anomalous.
Alert: 240 data point is anomalous.
Alert: 242 data point is anomalous.
Alert: 246 data point is anomalous.
Alert: 260 data point is anomalous.
Alert: 265 data point is anomalous.
Alert: 266 data point is anomalous.
Alert: 267 data point is anomalous.
Ale

score then learn (River):
Alert: 26 data point is anomalous.
Alert: 52 data point is anomalous.
Alert: 53 data point is anomalous.
Alert: 54 data point is anomalous.
Alert: 59 data point is anomalous.
Alert: 63 data point is anomalous.
Alert: 64 data point is anomalous.
Alert: 67 data point is anomalous.
Alert: 84 data point is anomalous.
Alert: 95 data point is anomalous.
Alert: 136 data point is anomalous.
Alert: 150 data point is anomalous.
Alert: 151 data point is anomalous.
Alert: 202 data point is anomalous.
Alert: 205 data point is anomalous.
Alert: 211 data point is anomalous.
Alert: 217 data point is anomalous.
Alert: 222 data point is anomalous.
Alert: 226 data point is anomalous.
Alert: 228 data point is anomalous.
Alert: 229 data point is anomalous.
Alert: 230 data point is anomalous.
Alert: 232 data point is anomalous.
Alert: 235 data point is anomalous.
Alert: 238 data point is anomalous.
Alert: 240 data point is anomalous.
Alert: 242 data point is anomalous.
Alert: 244 data point is anomalous.
Alert: 246 data point is anomalous.
Alert: 264 data point is anomalous.
Alert: 266 data point is anomalous.
Alert: 267 data point is anomalous.
Alert: 288 data point is anomalous.
Alert: 289 data point is anomalous.
Alert: 292 data point is anomalous.
Alert: 327 data point is anomalous.
Alert: 328 data point is anomalous.
Alert: 335 data point is anomalous.
Alert: 339 data point is anomalous.
Alert: 341 data point is anomalous.
Alert: 342 data point is anomalous.
Alert: 343 data point is anomalous.
Alert: 344 data point is anomalous.
Alert: 345 data point is anomalous.
Alert: 348 data point is anomalous.
Alert: 351 data point is anomalous.

learn then score (PySAD):
Alert: 24 data point is anomalous.
Alert: 51 data point is anomalous.
Alert: 52 data point is anomalous.
Alert: 53 data point is anomalous.
Alert: 67 data point is anomalous.
Alert: 76 data point is anomalous.
Alert: 83 data point is anomalous.
Alert: 126 data point is anomalous.
Alert: 136 data point is anomalous.
Alert: 150 data point is anomalous.
Alert: 205 data point is anomalous.
Alert: 211 data point is anomalous.
Alert: 217 data point is anomalous.
Alert: 222 data point is anomalous.
Alert: 226 data point is anomalous.
Alert: 228 data point is anomalous.
Alert: 229 data point is anomalous.
Alert: 232 data point is anomalous.
Alert: 234 data point is anomalous.
Alert: 235 data point is anomalous.
Alert: 238 data point is anomalous.
Alert: 240 data point is anomalous.
Alert: 242 data point is anomalous.
Alert: 246 data point is anomalous.
Alert: 249 data point is anomalous.
Alert: 266 data point is anomalous.
Alert: 276 data point is anomalous.
Alert: 326 data point is anomalous.
Alert: 327 data point is anomalous.
Alert: 328 data point is anomalous.
Alert: 330 data point is anomalous.
Alert: 333 data point is anomalous.
Alert: 334 data point is anomalous.
Alert: 335 data point is anomalous.
Alert: 338 data point is anomalous.
Alert: 339 data point is anomalous.
Alert: 340 data point is anomalous.
Alert: 341 data point is anomalous.
Alert: 342 data point is anomalous.
Alert: 343 data point is anomalous.
Alert: 344 data point is anomalous.
Alert: 347 data point is anomalous.
Alert: 348 data point is anomalous.

## PySAD version

In [18]:
from pysad.models import xStream
from pysad.transform.probability_calibration import ConformalProbabilityCalibrator
from pysad.utils import Data
import numpy as np
from pysad.evaluation import AUROCMetric

In [19]:
np.random.seed(61)

In [14]:
model = xStream()  # Init model.
calibrator = ConformalProbabilityCalibrator(windowed=True, window_size=300)  # Init probability calibrator.
streaming_data = Data().get_iterator("/Users/Sophie/Library/CloudStorage/OneDrive-Personnel/IP/Data Stream Processing/PySAD_to_River/data/arrhythmia.mat")  # Get streamer.

for i, (x, y_true) in enumerate(streaming_data):  # Stream data.
    anomaly_score = model.fit_score_partial(x)  # Fit to an instance x and score it.

    calibrated_score = calibrator.fit_transform(anomaly_score)  # Fit & calibrate score.

    # Output if the instance is anomalous.
    if calibrated_score > 0.95:  # If probability of being normal is less than 5%.
        print(f"Alert: {i}th data point is anomalous.")

Alert: 21th data point is anomalous.
Alert: 23th data point is anomalous.
Alert: 49th data point is anomalous.
Alert: 50th data point is anomalous.
Alert: 53th data point is anomalous.
Alert: 62th data point is anomalous.
Alert: 64th data point is anomalous.
Alert: 71th data point is anomalous.
Alert: 80th data point is anomalous.
Alert: 83th data point is anomalous.
Alert: 87th data point is anomalous.
Alert: 96th data point is anomalous.
Alert: 100th data point is anomalous.
Alert: 109th data point is anomalous.
Alert: 132th data point is anomalous.
Alert: 141th data point is anomalous.
Alert: 261th data point is anomalous.
Alert: 265th data point is anomalous.
Alert: 274th data point is anomalous.
Alert: 275th data point is anomalous.
Alert: 278th data point is anomalous.
Alert: 281th data point is anomalous.
Alert: 286th data point is anomalous.
Alert: 289th data point is anomalous.
Alert: 297th data point is anomalous.
Alert: 298th data point is anomalous.
Alert: 342th data point 

In [20]:
model = xStream()  # Init model
metric = AUROCMetric()  # Init area under receiver-operating- characteristics curve metric
streaming_data = Data().get_iterator("/Users/Sophie/Library/CloudStorage/OneDrive-Personnel/IP/Data Stream Processing/PySAD_to_River/data/arrhythmia.mat")  # Get data streamer.

for x, y_true in streaming_data:  # Stream data.
    anomaly_score = model.fit_score_partial(x)  # Fit the instance to model and score the instance.

    metric.update(y_true, anomaly_score)  # Update the AUROC metric.
    print(anomaly_score)
# Output the resulting AUROCMetric.
print(f"Area under ROC metric is {metric.get()}.")

[-2.]
[-2.]
[-2.00584963]
[-2.]
[-2.]
[-2.01754888]
[-2.0233985]
[-2.02924813]
[-2.06434588]
[-2.05094738]
[-2.093594]
[-2.08604513]
[-2.04094738]
[-2.02924813]
[-2.0801955]
[-2.08094738]
[-2.07604513]
[-2.04509775]
[-2.03924813]
[-2.05264663]
[-2.05264663]
[-2.11114288]
[-2.103594]
[-2.18454138]
[-1.09]
[-1.13]
[-1.1]
[-1.09]
[-1.08]
[-1.1]
[-1.08]
[-1.09]
[-1.11]
[-1.06]
[-1.08]
[-1.06]
[-1.05]
[-1.08]
[-1.1]
[-1.06]
[-1.09]
[-1.08]
[-1.1]
[-1.05]
[-1.08]
[-1.06]
[-1.09]
[-1.06]
[-1.1]
[-3.67734596]
[-3.94094981]
[-3.40187303]
[-3.25279121]
[-3.73461365]
[-3.53549014]
[-3.36491107]
[-3.31047051]
[-3.57915768]
[-3.59850612]
[-3.23142254]
[-3.47622088]
[-3.20505404]
[-3.7539417]
[-3.16939442]
[-3.75461842]
[-3.69606548]
[-3.68806606]
[-3.71403258]
[-3.74274588]
[-3.12774315]
[-3.49257854]
[-3.77722055]
[-3.66898274]
[-2.88425562]
[-3.05550567]
[-3.50893921]
[-3.1603925]
[-3.42829381]
[-3.08156402]
[-3.52544828]
[-3.98728939]
[-3.39114481]
[-3.41790586]
[-3.91699541]
[-3.66425115]
[-2.7

In [ ]:
#from pysad.evaluation import AUROCMetric
#from pysad.models import LODA
#from pysad.models import xStream
from pysad.utils import ArrayStreamer
from pysad.transform.ensemble import AverageScoreEnsembler
from pysad.utils import Data
from sklearn.utils import shuffle
from tqdm import tqdm
#import numpy as np
from scipy import io

In [ ]:
X_all, y_all = data.get_data("arrhythmia.mat")  # Load Aryhytmia data.
X_all, y_all = shuffle(X_all, y_all)  # Shuffle data.
iterator = ArrayStreamer(shuffle=False)  # Create streamer to simulate streaming data.
ROCAUC = metrics.ROCAUC()  # Tracker of area under receiver-operating- characteristics curve metric.

models = [  # Models to be ensembled.
    xStream(),
    LODA()
]
ensembler = AverageScoreEnsembler()  # Ensembler module.

for X, y in tqdm(iterator.iter(X_all, y_all)):  # Iterate over examples.
    model_scores = np.empty(len(models), dtype=np.float)

    # Fit & Score via for each model.
    for i, model in enumerate(models):
        model.fit_partial(X)
        model_scores[i] = model.score_partial(X)

    score = ensembler.fit_transform_partial(model_scores)  # fit to ensembler model and get ensembled score.

    auroc.update(y, score)  # update AUROC metric.

# Output score.
print("AUROC: ", auroc.get())

## Test avec notre adaptation River

In [22]:
from pysad.evaluation import AUROCMetric

In [23]:
model = xStream()     
metric = AUROCMetric()
for xi, yi in iter_array(X,y):
    x_array = np.array(list(xi.values()))
    y_array = np.array(list(yi.values()))
    anomaly_score = model.score_one(x_array)
    model = model.learn_one(x_array)
#    print(anomaly_score)
    ROCAUC.update(y_array[0],anomaly_score[0])
print(ROCAUC)

ROCAUC: -0.00%


In [24]:
model = xStream()
ROCAUC = metrics.ROCAUC()
for xi, yi in iter_array(X,y):
    x_array = np.array(list(xi.values()))
    y_array = np.array(list(yi.values()))
    anomaly_score = model.score_one(x_array)
    model = model.learn_one(x_array)
#    print(anomaly_score)
    ROCAUC.update(y_array[0],anomaly_score[0])
print(ROCAUC)

ROCAUC: -0.00%
